# PREPARATION

## Import Required Library

In [1]:
import math
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

## Global Helper Function

In [2]:
def create_criteria_decision_form() -> None: 
    global total_criteria
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias['type'] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            print(criterias)
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [3]:
def create_matrix_form() -> None:
    global total_criteria, total_alternative, criteria_labels, alternative_labels

    inputs = []
    for i in range(total_alternative):
        rows = []
        for c in range(total_criteria):
            rows.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
        inputs.append(rows)

    header_row = []
    for i in range(len(criteria_labels) + 1):
        header_row.append(widgets.Label(
            value='' if i == 0 else criteria_labels[i - 1],
            layout=widgets.Layout(width="100px")
        ))
    
    input_rows = []
    for r in range(total_alternative):
        row_widgets = [widgets.Label(value=alternative_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        
        decision_matrix.loc[:, :] = get_matrix()

        with output:
            print("Decision matrix:")
            print(decision_matrix)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

In [4]:
def create_matrix_weight() -> None:
    global total_criteria, criteria_labels

    inputs = []
    for i in range(total_alternative):
        inputs.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
    
    input_rows = []
    for r in range(total_criteria):
        row_widgets = [widgets.Label(value=criteria_labels[r], layout=widgets.Layout(width="100px"))] + [inputs[r]]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return [row.value for row in inputs]

    output = widgets.Output()
    
    def update_matrix(_):
        global weights
        
        weights['weight'] = get_matrix()

        with output:
            print("Weight Criteria:")
            print(weights)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Starter Matrix-Like Data Structure of Criterias and Alternatives

In [5]:
total_criteria = 5
total_alternative = 5

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

Define criterias

In [6]:
criterias = pd.DataFrame(None, index=criteria_labels, columns=['type'])
print(criterias)

   type
C1  NaN
C2  NaN
C3  NaN
C4  NaN
C5  NaN


Decide which criteria is benefit or cost

In [7]:
create_criteria_decision_form()

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Create matrix for decision

In [8]:
decision_matrix = pd.DataFrame(0, index=alternative_labels, columns=criteria_labels)

print(decision_matrix)

    C1  C2  C3  C4  C5
A1   0   0   0   0   0
A2   0   0   0   0   0
A3   0   0   0   0   0
A4   0   0   0   0   0
A5   0   0   0   0   0


In [9]:
create_matrix_form()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Normalization

Formula:

**Benefit criteria:**
$$
r_{ij} = \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

**Cost criteria:**
$$
r_{ij} = 1 - \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

In [10]:
matrix_r = decision_matrix.copy(deep=True).astype(float)

for i in range(total_criteria):
    sum_col = (decision_matrix.iloc[:, i] ** 2).sum()

    for j, row in enumerate(decision_matrix.itertuples(index=False)):
        if criterias.iloc[i]['type'] == 'Benefit':
            matrix_r.iloc[j, i] = decision_matrix.iloc[j, i] / math.sqrt(sum_col) # benefit criteria normalization
        else:
            matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization

print(matrix_r)

          C1        C2        C3        C4        C5
A1  0.592459  0.392857  0.619383  0.488306  0.556819
A2  0.762268  0.607143  0.429075  0.453427  0.433082
A3  0.354726  0.357143  0.651101  0.418548  0.494951
A4  0.592459  0.535714  0.429075  0.453427  0.402147
A5  0.558497  0.250000  0.714537  0.418548  0.309344


# Define Weights

In [11]:
weights = pd.DataFrame(0, index=criteria_labels, columns=['weight'])
print(weights)

    weight
C1       0
C2       0
C3       0
C4       0
C5       0


In [12]:
create_matrix_weight()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Calculate Weighted Matrix

Formula:

$$
v_{ij} = r_{ij} \cdot w_{ij}
$$

In [13]:
weighted_matrix = matrix_r.mul(weights['weight'], axis=1)
print(weighted_matrix)

          C1        C2        C3        C4         C5
A1  7.701962  6.285714  6.193831  6.836285  13.363666
A2  9.909478  9.714286  4.290747  6.347979  10.393962
A3  4.611440  5.714286  6.511012  5.859673  11.878814
A4  7.701962  8.571429  4.290747  6.347979   9.651536
A5  7.260459  4.000000  7.145373  5.859673   7.424259


# Get Concordance and Discordance Set

## Concordance

In [ ]:
print('Calculation should be here')

## Discordance

In [ ]:
print('Calculation should be here')

# Determine the Dominant Aggregate of The Matrix

In [ ]:
print('Calculation should be here')

# Eliminate Less Favourable Alternatives

In [ ]:
print('Calculation should be here')

# Rangking Alternative

In [ ]:
print('Calculation should be here')

---

# Conclusion